In [82]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 5
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

num_inputs = 28 * 28
"""展平每个图，看作784长的向量"""
num_outputs = 10
"""输出10个类别"""

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)


def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 广播


def net(X):
    """定义模型，图像展平"""
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)


# y每个元素代表一个样本，每个元素的值代表类别的索引
y = torch.tensor([0, 2])  # 第一个样本第一类是正确预测，第二个样本第三类是正确预测
# y_hat的一行代表一个样本，每列代表每个类别的概率
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]
"""第0行第0列和第1行第二列"""


def cross_entropy(y_hat, y):
    """交叉熵损失函数"""
    return -torch.log(y_hat[range(len(y_hat)), y])


def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = (
        y_hat.type(y.dtype) == y
    )  # type() 按输入的数据类型进行转换并返回   dtype 返回数据元素的数据类型（int、float等）
    return float(cmp.type(y.dtype).sum())


def evaluate_accuracy(net, data_iter):
    """计算再指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = d2l.Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


evaluate_accuracy(net, test_iter)


0.0905